# 1. Introduction

Amsterdam is the capital and most popular city of the Netherlands. It is colloquially referred to as the "Venice of the North", attributed by the large number of canals which form a UNESCO World Heritage Site(https://en.wikipedia.org/wiki/Amsterdam). Amsterdam is the heaven of art because of its high-density distribution of museums and art galleries. It has large amount of collections of Vincent Willem van Gogh and Rembrandt Harmenszoon van Rijn. Amsterdam is also famous of its open culture to sex and cannabis. According to https://www.dw.com/en/how-amsterdam-is-fighting-mass-tourism/a-47806959, there were 19 million tourists visiting Amsterdam in 2019 which brings the high demands of hotels.  

Airbnb is an online marketplace for arranging or offering lodging, primarily homestays, or tourism experiences (https://en.wikipedia.org/wiki/Airbnb). More and more people choose to stay in a local house when they are travelling. The prices of the houses vary a lot depending on the location, the size, the service or the surroundings of the houses.

So, in this project, I would like to use the data science techniques to solve the following question:

**Which factors of the property could affect the rental price on Airbnb?**

Which factors of the property could affect the rental price on Airbnb?
The answer of this question can be useful for people from Amsterdam who wants to start renting their properties on Airbnb, or tourists who are going to visit Amsterdam and want to estimate the cost of accommodation.

In [ ]:
import folium # map rendering library
import requests # library to handle requests
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

# 2. Data

The data we will use are:
* The data of Airbnb properties in Amsterdam
* The venue information provided by Foursquare API
* The coordinate of Amsterdam center provided by Google Map

In [ ]:
# The coordinates of Amsterdam center
latitude = 52.372952
longitude = 4.906080

The Airbnb data of Amsterdam was retrieved from Kaggle (https://www.kaggle.com/erikbruin/airbnb-amsterdam).The dataset includes the following files:
* calendar.csv: The calendar has 365 records for each listing. It specifies the whether the listing is available on a particular day (365 days ahead), and the price on that day.
* listings.csv: A listing is basically an advertisement. This file holds the most useful variables that can be used visualizations.
* listings_details.csv: This file holds the same variables as the listing file plus 80 additional variables.
* neighbourhood.csv: Simple file with the Dutch names of the neighbouhoods
* reviews.csv: This is a simple file that can be used to count the number of reviews by listing (for a specific period).
* reviews_details.csv: This file holds the full details of all reviews, and can also be used for instance for text mining.
* neighbourhoods.geojson: This is the shape file that can be used in conjunction with interactive maps (such as Leaflet for R of the Python folium package).

## 2.1 Airbnb Amsterdam data

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

We use the data from listings_details.csv to describ the airbnb properties.

In [ ]:
ab_data = pd.read_csv('/kaggle/input/airbnb-amsterdam/listings_details.csv')
print(ab_data.shape)
ab_data.head()

For each property in the dataset, it has following information:

In [ ]:
ab_data.columns

## 2.2 Amsterdam Neighbourhoods Coordinates

We use neighbourhoods.geojson to get the longitude and latitude of the center of each neighbourhood.

In [ ]:
import geopandas as gpd
geo_ams = gpd.read_file('/kaggle/input/airbnb-amsterdam/neighbourhoods.geojson')

In [ ]:
geo_ams["longitude"] = geo_ams.centroid.x
geo_ams["latitude"] = geo_ams.centroid.y
geo_ams.drop('neighbourhood_group', axis=1, inplace=True)
geo_ams

## 2.3 Foursquare data

We will use the FourSquare API (www.foursquare.com) to explore the venues of each neighbourhood in Amsterdam. Given a pair of coordinates, the Foursquare explore function can be used to retrieve the venues nearby. The categories and the number of venues can describe how conveninent living in a place.

# 3. Metrology

In this section, we will explore the data we use. Data cleaning and feature engineering methods will be used to prepare the data that is ready to fit into the model.

## 3.1 Airbnb data cleaning

There are 96 columns in the airbnb data. But there are columns not valuable for our analysis such as listing_url, scrape_id. So, we can firstly filter some columns that are not describing the property.

In [ ]:
ab_data.columns

In [ ]:
columns = ['id', 'name',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'square_feet',
       'price', 'cleaning_fee', 'guests_included', 'extra_people', 
       'minimum_nights', 'maximum_nights', 
       'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 
       'number_of_reviews', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy']

In [ ]:
data_columns = ab_data[columns]
print(data_columns.shape)

Now, there are 35 columns left.

### 3.1.1 Drop the unavaliable samples

We would like to focus on the properties that are available regularly. So, we can remove remove the properties are not availiable in the past recent months.

In [ ]:
data_columns = data_columns[data_columns['has_availability'] == 't']
data_columns.shape

In [ ]:
data_columns = data_columns[data_columns['availability_30'] != 0]
data_columns = data_columns[data_columns['availability_60'] != 0]
data_columns = data_columns[data_columns['availability_90'] != 0]
data_columns = data_columns[data_columns['availability_365'] != 0]
data_columns.shape

In [ ]:
data_columns.drop(['has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365'], axis=1, inplace=True)
data_columns.shape

### 3.1.2 Drop the samples missing information

Also, we would like to focus on the valid property only which means the property has a valid price and reviews from previous guests to verify the authenticity.

In [ ]:
data_columns = data_columns[data_columns['price'] != 0]
data_columns = data_columns[data_columns['number_of_reviews'] != 0]
data_columns = data_columns[data_columns['review_scores_rating'].notnull()]
data_columns.shape

### 3.1.3 Examine the missing values

Then, let's deal with the missing values in the data.

In [ ]:
def examine_missing_values(data):
    data_na= data.isnull().sum().sort_values(ascending=False)
    data_na_percent = (data.isnull().sum()/len(data)*100).sort_values(ascending=False)
    missing_data = pd.concat([data_na, data_na_percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

In [ ]:
missing_data = examine_missing_values(data_columns)
missing_data.head(20)

### Drop the feature having too many missing values

More than 97% of properties do not have the square feet recorded. So, we remove the square_feet from the data.

In [ ]:
data_columns.drop(['square_feet'], axis=1, inplace=True)
data_columns.shape

### Fill the missing sub-reviews by corresponding review_scores_rating

If a review score for a sub-catogery is missing, we fill it by its average review score which is review_score_rating.

In [ ]:
data_columns['review_scores_value'].fillna(data_columns['review_scores_rating'], inplace=True)
data_columns['review_scores_location'].fillna(data_columns['review_scores_rating'], inplace=True)
data_columns['review_scores_communication'].fillna(data_columns['review_scores_rating'], inplace=True)
data_columns['review_scores_checkin'].fillna(data_columns['review_scores_rating'], inplace=True)

### Fill the missing bathrooms, bedrooms, beds by 0

If the values of bathrooms, bedrooms, beds are missing, we can understand there is no bathroom, bedroom, bed in this property. So, we fill them by 0.

In [ ]:
data_columns['bathrooms'].fillna(0, inplace=True)
data_columns['bedrooms'].fillna(0, inplace=True)
data_columns['beds'].fillna(0, inplace=True)

### Fill the missing cleaning fee by mode

We fill the missing cleaning fee by mode.

In [ ]:
data_columns['cleaning_fee'].fillna(data_columns['cleaning_fee'].mode()[0], inplace=True)

### Examine missing values again

In [ ]:
missing_data = examine_missing_values(data_columns)
missing_data.head(20)

Now, there is no more missing value in our dataset.

In [ ]:
data_columns.shape

### 3.1.4 Convert the type of some features

In [ ]:
numeric_features = list(data_columns.dtypes[data_columns.dtypes != 'object'].index)
numeric_features.remove('id')
numeric_features.remove('latitude')
numeric_features.remove('longitude')
numeric_features

In [ ]:
category_features = list(data_columns.dtypes[data_columns.dtypes == 'object'].index)
category_features.remove('name')
category_features

We can see there are some features related to price that should be numerical rather than categorical, such as price, cleaning_fee, extra_people. That is because the money data is recorded as a string in the format of '50$'. So we convert these money-related features to numbers.

In [ ]:
data_columns['price'] = data_columns['price'].apply(lambda x:x.lstrip('$'))
data_columns['price'] = data_columns['price'].apply(lambda x:x.replace(',',''))
data_columns['price'] = data_columns['price'].astype('float')
data_columns['cleaning_fee'] = data_columns['cleaning_fee'].apply(lambda x:x.lstrip('$'))
data_columns['cleaning_fee'] = data_columns['cleaning_fee'].astype('float')
data_columns['extra_people'] = data_columns['extra_people'].apply(lambda x:x.lstrip('$'))
data_columns['extra_people'] = data_columns['extra_people'].astype('float')

We then add cleaning_fee and exra_people to numeric features list. Since price is our predicting value, we won't put it in the feature list.

In [ ]:
numeric_features.extend(['cleaning_fee', 'extra_people'])
numeric_features

Also, excluding the above features from the category features list.

In [ ]:
category_features.remove('price')
category_features.remove('cleaning_fee')
category_features.remove('extra_people')

We rename the neighbourhood_cleansed to neighbourhood for better understanding.

In [ ]:
data_columns.columns = ['neighbourhood' if x=='neighbourhood_cleansed' else x for x in data_columns.columns]
features = data_columns
print(features.shape)
features.head()

In [ ]:
category_features.remove('neighbourhood_cleansed')
category_features.append('neighbourhood')
category_features

### 3.1.5 Correct highly skewed numerical features

The skewed distribution has the following disadvantages:
* Highly skewed distributions are difficult to examine because most of the observations are confined to a small part of the range of the data.
* Outlying values in the direction of the skew are brought in toward the main body of the data when the distribution is made more symmetric. 

For the features with high skewness, I uesed box-cox transformation to correct the non-normal distribution while maintaining the information.

In [ ]:
from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p
skew_features = features[numeric_features].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

for i in skew_index:
    features[i] = boxcox1p(features[i], boxcox_normmax(features[i] + 1))

### 3.1.6 Get dummy categorical features

We use one hot encoder to encode the categorical features.

In [ ]:
def one_hot_encode(data, columns):
    onehot = pd.get_dummies(data[columns])
    onehot['id'] = data['id']
    # move id column to the first column
    fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
    onehot = onehot[fixed_columns]
    return onehot

In [ ]:
features_onehot = one_hot_encode(features, category_features)
features_onehot.head()

## 3.2 Examine the distribution of price

Let's examine the normality of the price distribution.

In [ ]:
from scipy import stats
from scipy.stats import norm
import matplotlib.pyplot as plt

def check_dist(price):
    print('Checking the statistical distribution of prices')
    print(price.describe())
    
    print('Fitting the prices into normal distribution')
    sns.distplot(price, fit=norm)

    # Get the fitted parameters used by the function
    (mu, sigma) = norm.fit(price)
    print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

    #Now plot the distribution
    plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
                loc='best')
    plt.ylabel('Frequency')
    plt.title('Price distribution')

    #Get also the QQ-plot
    fig = plt.figure()
    res = stats.probplot(price, plot=plt)
    plt.show()

In [ ]:
check_dist(features['price'])

We can the distribution of price is highly skewed. Also, there are several outlier when the price is above 2000. We firstly remove outliers and then correct the distribution the price by log function.

In [ ]:
features = features[features['price']<2000]

In [ ]:
def correct_dist(price):
    price = np.log1p(price)

    #Check the new distribution 
    sns.distplot(price , fit=norm);

    # Get the fitted parameters used by the function
    (mu, sigma) = norm.fit(price)
    print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

    #Now plot the distribution
    plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
                loc='best')
    plt.ylabel('Frequency')
    plt.title('Price distribution')

    #Get also the QQ-plot
    fig = plt.figure()
    res = stats.probplot(price, plot=plt)
    plt.show()
    return price

In [ ]:
features['corrected_price'] = correct_dist(features['price'])

## 3.3 Neighbourhoods Exploration
Firstly, let's see the number of properities of neighbourhoods.

In [ ]:
neighbourhood_count = pd.DataFrame({'neighbourhood': features['neighbourhood'].value_counts().index, 
                                    'count': features['neighbourhood'].value_counts().values})
neighbourhood_count.head()

In [ ]:
map_ams_count = folium.Map(location=[latitude, longitude], zoom_start=12)
map_ams_count.choropleth(
    geo_data=r'/kaggle/input/airbnb-amsterdam/neighbourhoods.geojson',
    data=neighbourhood_count,
    columns=['neighbourhood', 'count'],
    key_on='feature.properties.neighbourhood',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='The number of properites',
    reset=True
)
map_ams_count

The neighbourhood closer to the center, the more properites it has. The top 3 neighbourhoods are Oud-West, Centrum-West, De Pijp - Rivierenbuurt, Centrum-Oost, Zuid.

Then, let's see how the median of the prices distributied in different neighbourhoods.

In [ ]:
neighbourhood_price = features.groupby('neighbourhood').median()['price']
neighbourhood_price = pd.DataFrame({'neighbourhood':neighbourhood_price.index, 'price': neighbourhood_price.T.values})
neighbourhood_price.sort_values('price', ascending=False, inplace=True)
neighbourhood_price.head()

In [ ]:
map_ams_price = folium.Map(location=[latitude, longitude], zoom_start=12)
map_ams_price.choropleth(
    geo_data=r'/kaggle/input/airbnb-amsterdam/neighbourhoods.geojson',
    data=neighbourhood_price,
    columns=['neighbourhood', 'price'],
    key_on='feature.properties.neighbourhood',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='The average_price',
    reset=True
)
map_ams_price

Same, the neighbourhood closer to the center, the higher median of the prices it has. Top 2 neighbourhoods are Centrum-Oost, Centrum-West.

## 3.4 Venues Exploration

Foursquare requires client id, client secret and version for accessing their API.

In [ ]:
CLIENT_ID = 'H201PLIGMNTAP5ZKN2DZK1QSDVTSZNLH4SGVA0VBPFO00MFT' # your Foursquare ID
CLIENT_SECRET = 'JAPGNRKSTAJEQUKPATJJCNFETSEJQBAQRPDODZDNU1CN1MLP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Each venue provided by Foursquare belongs to a category and a sub_category, and each category has an unique ID (https://developer.foursquare.com/docs/build-with-foursquare/categories/).   
Foursquare has main 10 categories: 
* 'Arts & Entertainment': '4d4b7104d754a06370d81259', 
* 'College & University': '4d4b7105d754a06372d81259', 
* 'Event': '4d4b7105d754a06373d81259', 
* 'Food': '4d4b7105d754a06374d81259',
* 'Nightlife Spot': '4d4b7105d754a06376d81259',
* 'Outdoors & Recreation': '4d4b7105d754a06377d81259',
* 'Professional & Other Places': '4d4b7105d754a06375d81259',
* 'Residence': '4e67e38e036454776db1fb3a',
* 'Shop & Service': '4d4b7105d754a06378d81259',
* 'Travel & Transport': '4d4b7105d754a06379d81259'}


The sub_category gives more detailed description. For example, the type of restaurant, such as Chinese restaurant.  

In our case, the 10 main categories would be enough for us to understand the type of venues nearby. The more detailed categories are not very helpful for us to define a location. For example, we can say a place is suitable for people who has requirement for food if there are more than 20 restaurants nearby. It is not necessary to know whether there it's a Chinese restaurant or Italy restaurant.

There are three categories in the 10 main categories usually won't affect the choices of people when they are in travel: College \& University, Professional \& Other Places, Residence. So we won't consider the venues belonging to these there categories.

In [ ]:
# categories = {'Arts & Entertainment': '4d4b7104d754a06370d81259', 
#                   'College & University': '4d4b7105d754a06372d81259', 
#                   'Event': '4d4b7105d754a06373d81259', 
#                   'Food': '4d4b7105d754a06374d81259',
#                   'Nightlife Spot': '4d4b7105d754a06376d81259',
#                   'Outdoors & Recreation': '4d4b7105d754a06377d81259',
#                   'Professional & Other Places': '4d4b7105d754a06375d81259',
#                   'Residence': '4e67e38e036454776db1fb3a',
#                   'Shop & Service': '4d4b7105d754a06378d81259',
#                   'Travel & Transport': '4d4b7105d754a06379d81259'}
categories = {'Arts & Entertainment': '4d4b7104d754a06370d81259', 
                  'Event': '4d4b7105d754a06373d81259', 
                  'Food': '4d4b7105d754a06374d81259',
                  'Nightlife Spot': '4d4b7105d754a06376d81259',
                  'Outdoors & Recreation': '4d4b7105d754a06377d81259',
                  'Shop & Service': '4d4b7105d754a06378d81259',
                  'Travel & Transport': '4d4b7105d754a06379d81259'}

Given a pair of coordinate, Foursquare explore function can be used to return the venues nearby with the restriction of radius 1km and limit 100. In our case, we would like to know the number of venues of each main category founded near a location.

In [ ]:
def getNearbyVenues(data, categories, radius=500, limit=10):
    
    venues_list=[]
    print('Obtaining venues around the neighbourhoods: ', end='')
    for name, lat, lng in zip(data['neighbourhood'], data['latitude'], data['longitude']):
        print('.', end='')
        # create the API request URL
        venues = {'neighbourhood':name}
        for category, category_id in categories.items():
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                category_id,
                radius, 
                limit)

            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items'] 
            venues[category] = len(results)
        venues_list.append(venues)

    venues_list = pd.DataFrame(venues_list)
    return venues_list

Unfortunately, because there are too many properties in the data and Foursquare has its limitation for the number of daily requests, it is impossible to explore the venues of every property. Alternatively, I assume the venue distribution of properties in the same neighbourhood can be similar, so I use the data of each neighbourhood center to represent the properties belonging to each neighbourhood.

In [ ]:
neighbourhood_venues = getNearbyVenues(geo_ams, categories, radius=1000, limit=100)
print()
print(neighbourhood_venues.shape)
neighbourhood_venues.head()

We scale the number of venues of different categories by their percentage.

In [ ]:
neighbourhood_venues.iloc[:, 1:] = neighbourhood_venues.iloc[:, 1:].apply(lambda x: x/x.sum(), axis=1)
neighbourhood_venues.head()

In [ ]:
neighbourhood_venues.fillna(0, inplace=True)
neighbourhood_venues.head()

## 3.5 Merge Features

Now, we have all of the features we need. Let's merge all of them.

### Merge all the features we have now.

Merge the propoerty features with neighbourhood venues by their corresponding neighbourhood.

In [ ]:
features_merged = pd.merge(features, neighbourhood_venues)
features_merged.head()

Merge the features with dummy features by IDs of properties.

In [ ]:
features_merged_dummy = pd.merge(features_merged, features_onehot)
features_merged_dummy.head()

Drop the duplicate categorical features that have already been encoded.

In [ ]:
features_merged_dummy.drop(category_features, axis=1, inplace=True)
features_merged_dummy.head()

The price of the property is the target we are going to predict.
And remove all the unnecessary information of the properties,

In [ ]:
y = features_merged_dummy['corrected_price']
X = features_merged_dummy.drop(['id', 'name', 'latitude', 'longitude', 'price', 'corrected_price'], axis=1)
X.shape

### Drop overfitting features

Drop the features that have more than 95% same values for all the data.

In [ ]:
overfit = []
for i in X.columns:
    counts = X[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len(X) * 100 > 95:
        overfit.append(i)
overfit

In [ ]:
X.drop(overfit, axis=1, inplace=True)
X.shape

In [ ]:
X.columns

## 3.6 Regression Modelling

Given the information of the property, we would like to predict its price. In this section, we are going to fit our data into multiple common regression models and compare their performance on price prediction.  

To evaluate the performance of models, we firstly separate the data into training set and test set. The training set will be used to select and train the model, and the test set will be used to test the performance of the model. 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 3.6.1 Evaluation Matrix

We use the root of mean-squared-error (rmse) to measure how close the prediction to the actual price. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html.
The lower rmse, the model is better.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

# Root Mean Squared Logarithmic Error ，RMSLE
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

#
def cv_rmse(model):
    rmse = np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv=10))
    return (rmse)

### 3.6.2 Regression Models

We will fit out data into the following regression models:
* Lasso
* Ridge
* ElasticNet
* GradientBoostingRegressor
* XGBoosting regresoor
* LightGBM regressor

We use the model from sklearn. https://scikit-learn.org/stable/supervised_learning.html#supervised-learning

In [ ]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
#ridge
ridge = Ridge()

#lasso
lasso = Lasso()

#elastic net
elasticnet = ElasticNet()

#GradientBoosting
gbr = GradientBoostingRegressor(n_estimators=3000)


#lightgbm
lightgbm = LGBMRegressor(
    objective='regression',
    num_leaves=4,
    learning_rate=0.01,
    n_estimators=5000)

#xgboost（
xgb = XGBRegressor(learning_rate=0.01, 
                   booster='gbtree',
                   objective='reg:linear',
                   eval_metric='rmse',
                   max_depth=3,
                   min_child_weight=0,
                   n_estimators=3000)

### 3.6.3 Model Comparison
We use the cross validation to compare the performance of models,

In [ ]:
print('TEST score')

score = cv_rmse(ridge) 
print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()) ) 

score = cv_rmse(lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()) ) 

score = cv_rmse(elasticnet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = cv_rmse(lightgbm)
print("Lightgbm score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = cv_rmse(gbr)
print("GradientBoosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = cv_rmse(xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

We can see boosting models (Gradient boosting, xgboosting, lightgm) show much better results than the other models. Ridge also works great and Ridge runs much fast than boosting models. So we will not consider lasso and elastinet models anymore.

### 3.6.4 Model fitting

Let's fit the whole training data to the remaining models.

In [ ]:
print('START Fit')
print('ridge')
ridge_model_full_data = ridge.fit(X_train, y_train)
print('GradientBoosting')
gbr_model_full_data = gbr.fit(X_train, y_train)
print( 'xgboost')
xgb_model_full_data = xgb.fit(X_train, y_train)
print('lightgbm')
lgb_model_full_data = lightgbm.fit(X_train, y_train)

The rmsle scores on the training set.

In [ ]:
print('ridge', rmsle(y_train, ridge_model_full_data.predict(X_train)))
print('gbr', rmsle(y_train, gbr_model_full_data.predict(X_train)))
print('xgb', rmsle(y_train, xgb_model_full_data.predict(X_train)))
print('lgb', rmsle(y_train, lgb_model_full_data.predict(X_train)))

### 3.6.5 Model Prediction

To get a better and robust model, we linearly blend the models. We give higher weights to boosting models due to their good performance and robustness.

In [ ]:
def blend_models_predict(X):
    return ((0.1 * ridge_model_full_data.predict(X)) + 
            (0.3 * gbr_model_full_data.predict(X)) + 
            (0.3 * xgb_model_full_data.predict(X)) + 
            (0.3 * lgb_model_full_data.predict(X)) )
            
print('RMSLE score on train data:')
print(rmsle(y_train, blend_models_predict(X_train)))

Predict the prices of test set using trained blend_models

In [ ]:
y_pred = blend_models_predict(X_test)
rmsle(y_test, y_pred)

# 4. Results

The above rmsle score shows our model can accurately predict the price given the information data of the property. Once we have the trained model, more importantly, we would like to know which factor effect the price most. This can be done by examining the coef_ attribute of linear regression model or the feature_importances_ attribute of boosting regression model which represent the importance scores of all features.

In [ ]:
linear_model_list = {'ridge': ridge_model_full_data}
nonlinear_model_list = {'GradientBoosting': gbr_model_full_data,
                       'XGBoosting': xgb_model_full_data,
                       'Lightgbm': lgb_model_full_data}

In [ ]:
feature_importance = []
for model_name, model in linear_model_list.items():
    feature_importance.append(model.coef_)
for model_name, model in nonlinear_model_list.items():
    feature_importance.append(model.feature_importances_)
feature_importance = pd.DataFrame(feature_importance, columns=X_train.columns)
feature_importance.index = list(linear_model_list.keys()) + list(nonlinear_model_list.keys())
feature_importance

We would like to know the top 10 most important features given by the models.

In [ ]:
def return_most_important_features(features_importance, top):
    features_sorted = features_importance.sort_values(ascending=False)
    return features_sorted.index.values[0:top]

In [ ]:
features_sorted = []
for ind in np.arange(feature_importance.shape[0]):
    features_sorted.append(return_most_important_features(feature_importance.iloc[ind, :], top=10))

In [ ]:
def return_columns(top):
    indicators = ['st', 'nd', 'rd']

    columns = []
    for ind in np.arange(top):
        try:
            columns.append('{}{} Important Feature'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Important Feature'.format(ind+1))
    return columns

In [ ]:
features_sorted = pd.DataFrame(features_sorted, columns=return_columns(top=10))
features_sorted.index = feature_importance.index
features_sorted

According to the above table, there is no surprise that the number of accommodates, the number of bedrooms and the room type are top 3 most important features to the price. Bigger house can accommodate more people therefore can be more expensive. Other related factors are the number of beds and the number of bathrooms.

On the other hand, reviews also affect the price. People tend to book the property that has more reviews and higher reviews. The number of reviews can describe the opening years and the popularity of the property and the review scores can reflect the condition and reliability of the property. The two most important reviews are for checkin and location. 

Location is definitely another important factor to the price. The review scores for location can reflect the neighbourhood the property locates and the how convenience the public transportation nearby. Property locating in the centrum neighbourhood can be more expensive than the others. This can by verified by the previous visualization of the medians of the prices of different neighbourhoods.

In [ ]:
map_ams_price

We can see that it is true that the properties locating in Centrum-Oost and Centrum-West are more expensive than the others. It is worthy to mention these two neighbourhoods are the places where many famous places locate such as Anne Frank house, Red light district, Amsterdam museum. Also, it is interesting to see that IJburg - Zeeburgereiland also shows the high price even though it is not very close to the center. A reason could be the nice seaside scenery when living on the island. 

Back to the previous feature importance table, model Lightgbm shows a kind of different result than the other three models. It says cleaning_fee and fee for extra people can also effect the price. If the host doesnt charge for the fee for cleaning, that might means the fee has already included in the price. On the other hand, if the extra guest is allowed, the average cost per person might won't be very high even the fee for extra people is charged.

Next, let's look at the venues features only.

In [ ]:
venues_sorted = []
for model_name, model_feature_importance in feature_importance.iterrows():
    venues_sorted.append(return_most_important_features(model_feature_importance[list(categories.keys())], top=7))
venues_sorted = pd.DataFrame(venues_sorted, columns=return_columns(top=7))
venues_sorted.index = feature_importance.index
venues_sorted

We can find something that is interesting that whether there is a nightlife spot nearby can affect the price a most. A proper  interpretation could be that the nightlife spots usually locate the centrum neighbourhoods so that the location is the actual influential factor. 

It is not surprising that Food and Shop & Service are two important venues. Tourists might don't want to spend a lot of time on finding restaurants or shopping places. What really surprises me is that Travel & Transport is ranked as the least important feature by three models. If we compare the area of Amsterdam and other famous tourism cities in the world, for instance, new york, the area of New york is three times larger that Amsterdam. Amsterdam has intense public transportation network including bus, metro, tram, ship. And bicycling is a fashion way in the Netherlands to go out. So, people can always find a convenient way to go out no matter where they live.

# 5. Discussion

According to the our analysis, the price of the proper on Airbnb in Amsterdam mostly varies based on the number of accommodates/the number of bedrooms, the location and the extra fee (cleaning fee/fee for extra people). Bigger house and house close to city center or public transportation can be more expensive than the other. And the price can be higher if the extra fee is included in the price. Also, properties that locate near nightlife spots can be more expensive.

Here are my recommendations:

If you want to rent your house in Amsterdam on Airbnb:
* If you have a house in the popular neighbourhoods, then congratulations, you can rent your house in a good price.
* Ask for reviews from your guests. People trust on the previous review than your description and pictures. Showing on time when checking-in, being honest and providing good service (such as allowing extra guest) can really helpful on gaining higher review scores.

If you are going to visit Amsterdam and want to live in a local house:
* The more bedrooms and bathrooms in the house, the price can be more expensive. If you can find more travelling partners who don't mind sharing bedrooms or bathrooms with you, the average price per person can be very fair.
* It is always expensice to live in the center. If you really care about living close to center, I recommend to live in the neighbourhood of Oud-West which has the most number of houses but the median price ranks the 5th expensive neighbourhood. If you have an empty wallet, considering the mature public transportation network of Amsterdam, I recommend to live in the neighourhood far away from the center. Even if you live the suburb, a less than 30-minutes train can take you to the center.
* Places having many restaurants and shopping places nearby can be expensive. You can choose other places if you don't necessarily eat or shop near the place you live.
* If you are a night owl and a lover to night clubs of Amsterdam, unfortunately, you might need to prepare more budgets on accommodation.

There are might be other factors can effect the price such as the decoration of the property and the service the host can provide. Unfortunately, we can't go through the influence of all of possible factors due to the unavailability of the data. However, I believe my analysis is extremely valuable for the hosts from Amsterdam who wants to define the price of their proprieties or tourists who want to have a estimate of the accommodation cost based on their requirements and budgets. This analysis can also be applied to any other city in the world.

# 6. Conclusion

In this project, I analyzed the Airbnb houses in Amsterdam using the Airbnb data and the venue information of neighbourhoods provided by Foursquare API. I used several classic regression models to predict the price of the properties and analyzed the most important factors to the price of a property. Based on the above analysis, several recommendations were proposed to people who want to rent their house on Airbnb and people who to live in a local place in Amsterdam. This analysis can be adapted to any other city's Airbnb house price analysis.

This is the final project of my IBM professional certificate course on Coursera. I would like to thanks to myself for consisting motivating myself to explore better solution, and I also would like to thanks to the time of the person who is reviewing this work. Hope everyone can by happy and safe.